In [1]:
import glob
# 2. 기계학습으로 감성분석(시간이 매우 오래 걸림)
#긍정 텍스트 로딩
pos_review=(glob.glob("c:/data/imdb/train/pos/*.txt")[:100]
)
lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue
len(lines_pos)

100

In [2]:
#부정 텍스트 로딩
neg_review=(glob.glob("c:/data/imdb/train/neg/*.txt")[:100]
)
lines_neg=[]
for i in neg_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except Exception as e:
        continue
len(lines_neg)
# 긍정, 부정 리뷰를 합침
total_text=lines_pos+lines_neg
len(total_text)

200

In [3]:
import numpy as np
from nltk.corpus import stopwords
# 긍정, 부정 클래스 라벨링
x = np.array(["pos", "neg"])
class_Index=np.repeat(x, [len(lines_pos), len(lines_neg)], axis=0)
stop_words = stopwords.words('english')

In [4]:
# 단어들에 Tfidf 가중치를 부여한 후 문서 - 단어 매트릭스로 바꿈
from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf 계산
vect = TfidfVectorizer(stop_words=stop_words).fit(total_text)
X_train_vectorized = vect.transform(total_text)
X_train_vectorized.index = class_Index


In [5]:
import pandas as pd
# 데이터프레임으로 변환
df=pd.DataFrame(X_train_vectorized.toarray(), columns=vect.vocabulary_.keys())
df.head()

,bromwell,high,cartoon,comedy,ran,time,programs,school,life,teachers,...,zombified,auteur,ample,opportunities,golden,geist,uttered,downloading,midget,tricking
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#로지스틱 회귀 모형
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=10)
logit.fit(X_train_vectorized, class_Index)

LogisticRegression(random_state=10)

In [7]:
#긍정 리뷰들을 하나씩 불러와서 실험
def pos_review(model):
    count_all=0
    count=0
    num=100
    tests1=[] # 리스트
    for idx in range(0,num):
        pos_review_test=(glob.glob("c:/data/imdb/test/pos/*.txt"))[idx]
        f = open(pos_review_test, 'r',encoding="utf-8")
        tests1.append(f.readlines())
        f.close()
    for test in tests1:
        preds = model.predict(vect.transform(test))
        result=preds[0]
        if result=="pos":
            count+=1
        count_all += 1
    rate= count*100/count_all
    print(" 예측정확도:{0:.1f}%".format(rate))

In [8]:
#부정 리뷰들을 하나씩 불러와서 실험
def neg_review(model):
    count_all=0
    count=0
    num=100
    tests2=[]
    for idx in range(0,num):
        neg_review_test=(glob.glob("c:/data/imdb/test/neg/*.txt"))[idx]
        f = open(neg_review_test, 'r',encoding="utf-8")
        tests2.append(f.readlines())
        f.close()
    for test in tests2:
        preds = model.predict(vect.transform(test))
        result=preds[0]
        if result=="neg":
            count+=1
        count_all+=1
    rate= count*100/count_all
    print(" 예측정확도:{0:.1f}%".format(rate)) 

In [24]:
pos_review(logit)
neg_review(logit)

 예측정확도:66.0%
 예측정확도:82.0%


In [10]:
#의사결정나무 모형
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=10)
tree.fit(X_train_vectorized, class_Index)

DecisionTreeClassifier(random_state=10)

In [25]:
pos_review(tree)
neg_review(tree)

 예측정확도:54.0%
 예측정확도:54.0%


In [12]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
#10개의 트리로 구성된 랜덤 포레스트
forest = RandomForestClassifier(n_estimators=10, random_state=10)
forest.fit(X_train_vectorized, class_Index)


RandomForestClassifier(n_estimators=10, random_state=10)

In [23]:
pos_review(forest)
neg_review(forest)

 예측정확도:46.0%
 예측정확도:70.0%


In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train_vectorized, class_Index)

KNeighborsClassifier(n_neighbors=2)

In [22]:
pos_review(knn)
neg_review(knn)

 예측정확도:34.0%
 예측정확도:85.0%


In [16]:
#인공신경망
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=10)
mlp.fit(X_train_vectorized, class_Index)


MLPClassifier(random_state=10)

In [17]:
pos_review(mlp)
neg_review(mlp)


 예측정확도:63.0%
 예측정확도:76.0%


In [18]:
#SVM 모형
from sklearn.svm import SVC
svm = SVC(random_state=10)
svm.fit(X_train_vectorized, class_Index)

SVC(random_state=10)

In [19]:
pos_review(svm)
neg_review(svm)

 예측정확도:63.0%
 예측정확도:86.0%
